# FedNAMs+ Full Pipeline Test

End-to-end test of the complete FedNAMs+ pipeline with 10K NIH images.

## What This Tests:
1. Data loading from Google Drive
2. Federated data partitioning
3. Model creation (FedNAMs)
4. Federated training
5. Evaluation
6. Results saving

In [ ]:
# === CELL 1: Setup ===
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content')

# Clone/update repo
if not os.path.exists('fednams-plus'):
    !git clone https://github.com/siddharth10ss/fednams-plus.git
else:
    %cd fednams-plus
    !git pull origin main
    %cd ..

%cd fednams-plus

# Install dependencies
!pip install -q torch torchvision pandas numpy pillow pyyaml tqdm

print("✓ Setup complete")

In [ ]:
# === CELL 2: Import and Configure ===
import sys
from pathlib import Path
import torch

# Add project to path
sys.path.insert(0, '/content/fednams-plus')

from configs.config import (
    PreprocessConfig,
    ModelConfig,
    TrainingConfig,
    FedConfig,
    ExperimentConfig
)
from experiments import ExperimentRunner

print("✓ Imports successful")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# === CELL 3: Create Configuration ===

# Data config
data_config = PreprocessConfig(
    image_size=(224, 224),
    normalization='imagenet',
    augmentation=True,
    augmentation_params={
        'horizontal_flip_prob': 0.5,
        'rotation_degrees': 10,
        'brightness': 0.2,
        'contrast': 0.2
    }
)

# Model config
model_config = ModelConfig(
    backbone='resnet18',
    pretrained=True,
    feature_dim=512,
    num_classes=15,  # NIH has 15 classes
    nam_hidden_units=[64, 32],
    dropout=0.3,
    use_exu=False
)

# Training config (reduced for quick test)
training_config = TrainingConfig(
    batch_size=16,  # Smaller for Colab
    learning_rate=0.001,
    num_local_epochs=2,  # Reduced for testing
    optimizer='adam',
    scheduler='cosine',
    early_stopping_patience=5,
    mixed_precision=True
)

# Federated config (reduced for quick test)
fed_config = FedConfig(
    num_clients=3,  # Fewer clients for testing
    num_rounds=5,   # Fewer rounds for testing
    client_fraction=1.0,
    min_clients=2
)

# Complete experiment config
experiment_config = ExperimentConfig(
    experiment_name='fednams_pipeline_test',
    data_config=data_config,
    model_config=model_config,
    training_config=training_config,
    fed_config=fed_config,
    output_dir=Path('/content/drive/MyDrive/FedNAMs_Results/pipeline_test'),
    seed=42,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

print("✓ Configuration created")
print(f"  - Device: {experiment_config.device}")
print(f"  - Clients: {fed_config.num_clients}")
print(f"  - Rounds: {fed_config.num_rounds}")
print(f"  - Batch size: {training_config.batch_size}")

In [ ]:
# === CELL 4: Initialize Experiment Runner ===

runner = ExperimentRunner(experiment_config)

print("✓ ExperimentRunner initialized")
print(f"  - Output dir: {experiment_config.output_dir}")

In [ ]:
# === CELL 5: Run Complete Experiment ===

# Path to your NIH dataset
data_dir = Path('/content/drive/MyDrive/NIH Chest XRAY Dataset')

print("🚀 Starting full pipeline test...")
print("="*70)
print("\nThis will:")
print("  1. Load 10K NIH images from Google Drive")
print("  2. Partition data across 3 federated clients")
print("  3. Create FedNAMs+ model (ResNet18 + NAM)")
print("  4. Train for 5 federated rounds")
print("  5. Evaluate on test set")
print("  6. Save results to Google Drive")
print("\n" + "="*70)
print("\n⏳ This will take approximately 15-20 minutes...\n")

# Run experiment
results = runner.run_experiment(data_dir=data_dir)

print("\n" + "="*70)
print("✅ EXPERIMENT COMPLETE!")
print("="*70)

In [ ]:
# === CELL 6: Display Results ===

print("\n📊 RESULTS SUMMARY")
print("="*70)

if results['status'] == 'completed':
    print("\n✅ Status: SUCCESS")
    
    # Test metrics
    test_metrics = results['test_metrics']
    print("\n📈 Test Set Performance:")
    print(f"  - Accuracy:  {test_metrics['accuracy']:.4f}")
    print(f"  - F1-Score:  {test_metrics['f1']:.4f}")
    print(f"  - AUC-ROC:   {test_metrics['auc_roc']:.4f}")
    print(f"  - AUC-PR:    {test_metrics['auc_pr']:.4f}")
    
    # Training history
    history = results['training_history']
    print("\n📉 Training Progress:")
    print(f"  - Rounds completed: {len(history['rounds'])}")
    print(f"  - Final train loss: {history['train_loss'][-1]:.4f}")
    print(f"  - Final val loss:   {history['val_loss'][-1]:.4f}")
    print(f"  - Final train acc:  {history['train_accuracy'][-1]:.4f}")
    print(f"  - Final val acc:    {history['val_accuracy'][-1]:.4f}")
    
    print(f"\n💾 Results saved to: {experiment_config.output_dir}")
    
else:
    print("\n❌ Status: FAILED")
    print(f"Error: {results.get('error', 'Unknown error')}")

print("\n" + "="*70)

In [ ]:
# === CELL 7: Save Results ===

runner.save_results(results)

print("✓ Results saved to Google Drive")
print(f"\n📁 Check your results at:")
print(f"   {experiment_config.output_dir}")
print(f"\n📄 Files created:")
print(f"   - config.json (experiment configuration)")
print(f"   - results/results.json (metrics and history)")
print(f"   - experiment.log (detailed logs)")

## Summary

This notebook tested the complete FedNAMs+ pipeline:

✅ **Data Module**
- Loaded 10,000 NIH chest X-rays
- Applied preprocessing and augmentation
- Partitioned across federated clients

✅ **Model Module**
- Created FedNAMs model (ResNet18 + NAM head)
- Initialized with pretrained weights

✅ **Training Module**
- Federated training across multiple clients
- FedAvg aggregation
- Validation monitoring

✅ **Evaluation**
- Test set metrics
- Training history tracking

✅ **Results**
- Saved to Google Drive
- JSON format for easy analysis

### Next Steps:
1. Run longer training (more rounds/epochs)
2. Add SHAP explanations
3. Add conformal prediction
4. Compare with baselines